# Introduction

+ A partir de la modélisation vectorielle , grâce à la méthode des Kmeans , on déterminera 2 clusters dont l'un representera les mots à polarité positive et l'autre les mots à polarité négative.
+ On pourra dès lors prédire la polarité =  (1 ou -1 dépendement de sa nature positive ou négative) x l'inverse de sa proximité par rapport au centre du cluster.

On utilise une classification non supervisée car de base on ne connait pas la polarité des mots et c'est à cette dernière que l'on veut déterminer.

## Importation

In [ ]:
!pip install --upgrade gensim
import numpy as np
import pandas as pd
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## creation_cluster

+ Permet de créer des cluster à partir du model formé dans le fichier embedding.ipynb

+ Etant impossible de connaître quel indice de cluster correspond au sentiment positif ou  négatif , on affiche leurs 20 plus proches voisins et on détermine manuellement quel indice correspond au sentiments positis et negatifs.

In [ ]:
def creation_cluster():

  word_vectors = Word2Vec.load("../word2vec/word2vec.model").wv
  model = KMeans(n_clusters=2, max_iter=10000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

  centre_1=word_vectors.similar_by_vector(model.cluster_centers_[1], topn=20, restrict_vocab=None)
  centre_0=word_vectors.similar_by_vector(model.cluster_centers_[0], topn=20, restrict_vocab=None)

  print("pour les 20 mots plus proches du centre indexé à 1:  {}".format([word[0] for word in centre_1]))
  print("                                               à 0:  {}".format([word[0] for word in centre_0]))
  return model,word_vectors



In [ ]:
model,word_vectors=creation_cluster()

pour les 20 mots plus proches du centre indexé à 1:  ['fastidieux', 'est-elle', 'écocid', 'xxèm', 'socio-économique', 'électrification', 'harmoniser', 'disposez', 'éventualité', 'récréatif', 'suppléer', 'restrictif', 'aquatique', 'octroi', 'obstruction', 'quesque', 'consacre', 'sytème', 'responsabilisation', 'sédentarité']
                                               à 0:  ['multiverse', 'balécouill', 'ninporte', 'Insta', 'multivers', 'flair', 'Blc', 'Tageul', 'Marci', 'benmachin', 'Meilleure', 'est-çe', 'debilos', 'POV', 'nonvaccinophobie', 'Goat', 'MashAllah', 'idea', 'zipette', 'Influenceur']


##### On détermine le sentiment par indice

In [ ]:
#je construit la liste d'indices positives,negatives et neutre  de chaque moi moi en observant 20 plus proches voisins je deduis la nature des sentiments de chaque indice
pos=1
neg=0

## creation_dictionnaire_sentiment

Cette fonction permet de:
+ Créer un dictionnaire contenant l'ensemble des mots du vocabulaire et leur polarité
+ La polarité est déterminé par la prediction du sentiment du mot et son poids qui correspond à l'inverse de la distance au centre du cluster

In [ ]:
def creation_dictionnaire_sentiment(model,word_vectors,pos,neg):#,neu):

    positive_cluster_center = model.cluster_centers_[pos]
    negative_cluster_center = model.cluster_centers_[neg]
    #neutral_cluster_center = model.cluster_centers_[neu]

    words = pd.DataFrame(word_vectors.index_to_key)
    words.columns = ['words']
    words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
    words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
    words.cluster = words.cluster.apply(lambda x: x[0])

    words['cluster_value'] = [1 if i==pos else -1 for i in words.cluster]
    words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
    words['sentiment_coeff'] = words.closeness_score * words.cluster_value

    words[['words', 'sentiment_coeff']].to_csv("dict_sentiment.csv")
    return words[['words', 'sentiment_coeff']]

#### Construction du dictionnaire de sentiment

In [ ]:
creation_dictionnaire_sentiment(model,word_vectors,pos,neg)

,words,sentiment_coeff
0,faire,1.111830
1,le,1.161597
2,tout,1.147832
3,plus,1.178380
4,vous,1.176877
...,...,...
24798,descrimination,-2.125025
24799,aneanti,-2.618446
24800,tribun,-2.594583
24801,déporte,1.733132
